In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input,Dense,Conv2D,Add
from tensorflow.keras.layers import SeparableConv2D,ReLU
from tensorflow.keras.layers import BatchNormalization,MaxPool2D
from tensorflow.keras.layers import GlobalAvgPool2D
from tensorflow.keras import Model
import numpy
import netCDF4 as nc
import xarray
import tensorflow_datasets as tfds
import os
import shutil
import glob
#import plotly.express as px
from tensorflow import keras
import pandas as pd
import csv
from tensorflow.keras.metrics import Precision
from tensorflow.keras.metrics import AUC
import matplotlib.pyplot as plt


C:\Users\nafan\.conda\envs\test2\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\nafan\.conda\envs\test2\lib\site-packages\ecmwflibs\__init__.py:75: UserWarning: DLL load failed while importing _ecmwflibs: The specified procedure could not be found.
  warnings.warn(str(e))
C:\Users\nafan\.conda\envs\test2\lib\site-packages\xarray\backends\cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(
C:\Users\nafan\.conda\envs\test2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
dir_loc = 'N:/hpc/Data/'

In [6]:
def simple_load_nc_dir_with_generator(data, generate_tensor=True):
    #def gen():
        results_list = list()
        final_dict = None
        file_path = dir_loc + data +'.nc'
        #for file in glob.glob(os.path.join(dir_, "*.nc")):
        ds = xarray.open_dataset(file_path, engine='netcdf4')

        if generate_tensor == True:
            #final_dict = {'conv2d_input': results_list.append(tf.convert_to_tensor(val)) for key, val in ds.items()}
            #results_list = [tf.convert_to_tensor(val) for key, val in ds.items()]
            results_list += list(map( lambda x : tf.convert_to_tensor(x[1]) ,
                                     ds.items() ))
        else:
            #final_dict = {'conv2d_input': results_list + [val] for key, val in ds.items()}
            results_list += list(map( lambda x : x[1],
                                     ds.items() ))

        print("CCC", ds.items())

        results_list_shape = results_list[0].shape   
        final_dict = { 'input_1': tf.reshape( results_list[0:1],
                                               shape=(results_list_shape[0],
                                                      results_list_shape[1], 
                                                      results_list_shape[2], 1)) }

        print("X # of inputs:", final_dict['input_1'].shape, "and", type(final_dict['input_1']))

        return final_dict

X_values = simple_load_nc_dir_with_generator("x_train", generate_tensor=True)
print(type(X_values))
X_val = simple_load_nc_dir_with_generator("x_val", generate_tensor=True)
X_test = simple_load_nc_dir_with_generator("x_test", generate_tensor=True)


C:\Users\nafan\.conda\envs\test2\lib\site-packages\ecmwflibs\__init__.py:75: UserWarning: DLL load failed while importing _ecmwflibs: The specified procedure could not be found.
  warnings.warn(str(e))
C:\Users\nafan\.conda\envs\test2\lib\site-packages\xarray\backends\plugins.py:61: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


CCC ItemsView(<xarray.Dataset>
Dimensions:  (time: 31, lon: 481, lat: 281)
Coordinates:
  * time     (time) datetime64[ns] 2007-05-01 2007-05-02 ... 2007-05-31
  * lon      (lon) float64 -80.0 -79.75 -79.5 -79.25 ... 39.25 39.5 39.75 40.0
  * lat      (lat) float64 20.0 20.25 20.5 20.75 21.0 ... 89.25 89.5 89.75 90.0
Data variables:
    Z500     (time, lat, lon) float64 -15.08 -14.8 -14.53 ... -18.04 -18.04)
X # of inputs: (31, 281, 481, 1) and <class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'dict'>
CCC ItemsView(<xarray.Dataset>
Dimensions:  (time: 31, lon: 481, lat: 281)
Coordinates:
  * time     (time) datetime64[ns] 2008-05-01 2008-05-02 ... 2008-05-31
  * lon      (lon) float64 -80.0 -79.75 -79.5 -79.25 ... 39.25 39.5 39.75 40.0
  * lat      (lat) float64 20.0 20.25 20.5 20.75 21.0 ... 89.25 89.5 89.75 90.0
Data variables:
    Z500     (time, lat, lon) float64 -13.07 -13.15 -13.26 ... -48.53 -48.53)
X # of inputs: (31, 281, 481, 1) and <class 'tensorflow.python.framew

In [7]:
def open_csv_file(data):
   
    Y_values = list()
    with open(dir_loc + data) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                print(f'Column names are {", ".join(row)}')
                line_count += 1
            else:
                #print(f'\t{row[0]} and {row[1]} and {row[2]}.')
                if row[1] == "Yes":
                    Y_values.append(1)
                else:
                    Y_values.append(0)

                line_count += 1
        print(f'Processed {line_count} lines.')


    Y_values = list(map(lambda x : tf.convert_to_tensor(x), Y_values))

    Y_values = {'dense': tf.convert_to_tensor( Y_values ) }


    return Y_values
Y_values = open_csv_file("y_train.csv") 
Y_val = open_csv_file("y_val.csv") 
Y_test = open_csv_file("y_test.csv") 

Column names are , extreme, year, ex
Processed 32 lines.
Column names are , extreme, year, ex
Processed 32 lines.
Column names are , extreme, year, ex
Processed 32 lines.


In [8]:
# creating the Conv-Batch Norm block
#This part is partially taken 
#from https://towardsdatascience.com/xception-from-scratch-using-tensorflow-even-better-than-inception-940fb231ced9

def conv_bn(x, filters, kernel_size, strides=1):
    
    x = Conv2D(filters=filters, 
               kernel_size = kernel_size, 
               strides=strides, 
               padding = 'same', 
               use_bias = False)(x)
    x = BatchNormalization()(x)
    return x
# creating separableConv-Batch Norm block

def sep_bn(x, filters, kernel_size, strides=1):
    
    x = SeparableConv2D(filters=filters, 
                        kernel_size = kernel_size, 
                        strides=strides, 
                        padding = 'same', 
                        use_bias = False)(x)
    x = BatchNormalization()(x)
    return x
# entry flow

def entry_flow(x):
    
    x = conv_bn(x, filters =32, kernel_size =3, strides=2)
    x = ReLU()(x)
    x = conv_bn(x, filters =64, kernel_size =3, strides=1)
    tensor = ReLU()(x)
    
    x = sep_bn(tensor, filters = 128, kernel_size =3)
    x = ReLU()(x)
    x = sep_bn(x, filters = 128, kernel_size =3)
    x = MaxPool2D(pool_size=3, strides=2, padding = 'same')(x)
    
    tensor = conv_bn(tensor, filters=128, kernel_size = 1,strides=2)
    x = Add()([tensor,x])
    
    x = ReLU()(x)
    x = sep_bn(x, filters =256, kernel_size=3)
    x = ReLU()(x)
    x = sep_bn(x, filters =256, kernel_size=3)
    x = MaxPool2D(pool_size=3, strides=2, padding = 'same')(x)
    
    tensor = conv_bn(tensor, filters=256, kernel_size = 1,strides=2)
    x = Add()([tensor,x])
    
    x = ReLU()(x)
    x = sep_bn(x, filters =728, kernel_size=3)
    x = ReLU()(x)
    x = sep_bn(x, filters =728, kernel_size=3)
    x = MaxPool2D(pool_size=3, strides=2, padding = 'same')(x)
    
    tensor = conv_bn(tensor, filters=728, kernel_size = 1,strides=2)
    x = Add()([tensor,x])
    return x
# middle flow

def middle_flow(tensor):
    
    for _ in range(8):
        x = ReLU()(tensor)
        x = sep_bn(x, filters = 728, kernel_size = 3)
        x = ReLU()(x)
        x = sep_bn(x, filters = 728, kernel_size = 3)
        x = ReLU()(x)
        x = sep_bn(x, filters = 728, kernel_size = 3)
        x = ReLU()(x)
        tensor = Add()([tensor,x])
        
    return tensor
# exit flow

def exit_flow(tensor):
    
    x = ReLU()(tensor)
    x = sep_bn(x, filters = 728,  kernel_size=3)
    x = ReLU()(x)
    x = sep_bn(x, filters = 1024,  kernel_size=3)
    x = MaxPool2D(pool_size = 3, strides = 2, padding ='same')(x)
    
    tensor = conv_bn(tensor, filters =1024, kernel_size=1, strides =2)
    x = Add()([tensor,x])
    
    x = sep_bn(x, filters = 1536,  kernel_size=3)
    x = ReLU()(x)
    x = sep_bn(x, filters = 2048,  kernel_size=3)
    x = GlobalAvgPool2D()(x)
    
    x = Dense (units = 1, activation = 'sigmoid')(x)
    
    return x
# model code

METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]



input = Input(shape = (281, 481, 1))
x = entry_flow(input)
x = middle_flow(x)
output = exit_flow(x)

model = Model(inputs=input, outputs=output)
model.summary()

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=METRICS)
    
history = model.fit(
            X_values,
            Y_values,
            epochs=1,
            validation_data=(X_val, Y_val),
            batch_size=10
            )
# Fit the model
model.evaluate(X_test, Y_test)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 281, 481, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 141, 241, 32  288         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 141, 241, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                             

[0.6075934767723083,
 0.0,
 0.0,
 29.0,
 2.0,
 0.9354838728904724,
 0.0,
 0.0,
 0.5,
 0.06451612710952759]